In [2]:
import pandas as pd
dataset=pd.read_csv('./export/dataset.csv',index_col=0)

In [322]:
df=pd.read_csv('./export/q2_2往期品类销量定价关系汇总.csv')
(df['销量(千克)_否']/(df['销量(千克)_是']+df['销量(千克)_否'])).mean()

0.9380456720665777

In [323]:
import pandas as pd
from sympy import *
df=pd.read_csv('./export/q2_5品类数学期望和多项式回归数据总表.csv',index_col=0)

result=pd.DataFrame(columns=['分类名称','R_max','W','P','Q','S(Q)'])
waste_data=pd.read_excel('./dataset/附件4.xlsx',sheet_name='平均损耗率(%)_小分类编码_不同值',index_col='分类名称')['平均损耗率(%)_小分类编码_不同值']

for typ in df['分类名称'].values:
    # 导入常数
    df0=df[df['分类名称']==typ].copy()
    ec,ec2,ec3=df0['E(C)'].values[0]*1.1,df0['E(C^2)'].values[0]*1.1**2,df0['E(C^3)'].values[0]*1.1**3
    k1,k2,k3=df0['k1'].values[0],df0['k2'].values[0],df0['k3'].values[0]
    waste=waste_data[typ]/100
    
    # 定义函数
    w=symbols('w')
    r=k1*ec3*w*(1+w)**2+k2*ec2*(1+w)+k3*ec*w
    
    # 求导&求解
    dr=diff(r,w)
    solutions=solve(dr,w,real=True)
    
    # 计算
    w_max=max(solutions,key=lambda x: re(x))
    if im(w_max) != 0:
        r_max=r.subs(w,3) if r.subs(w,3)>=r.subs(w,0) else r.subs(w,0)
        w_max=3 if r.subs(w,3)>=r.subs(w,0) else 0
    else:
        r_max=r.subs(w,w_max)
    p=ec*(1+w_max)
    q=k1*p**2+k2*p+k3
    sq=q/(1-waste)
    
    # 打表
    row={'分类名称':typ,'R_max':r_max,'W':w_max,'P':p,'Q':q,'S(Q)':sq}
    result=pd.concat([result,pd.DataFrame(row,index=[0])],axis=0,ignore_index=True)

result

,分类名称,R_max,W,P,Q,S(Q)
0,水生根茎类,519.521654747076,0.359908549082561,6.69306190601964,61.7280774372778,71.4859032278840
1,花叶类,10082.8638745418,3,10.8004,243.134987,278.920485
2,花菜类,404.416358421956,2.71658795095135,16.4704311634360,39.5803159791312,46.8461545498061
3,茄类,2630.43080361493,3,16.1016,35.193163,37.712348
4,辣椒类,6373.86166310071,3,18.732,100.551605,110.788459
5,食用菌,414.788814170030,1.01768558035257,10.9231444263547,110.185502501878,121.684707346083


In [325]:
result2=pd.read_csv('./export/q2_6问题二最终结论.csv',index_col=0)
result2

,分类名称,R_max,W,P,Q,S(Q)
0,水生根茎类,519.521655,0.359909,6.693062,61.728077,71.485903
1,花叶类,10082.863875,3.000000,10.800400,243.134987,278.920485
2,花菜类,404.416358,2.716588,16.470431,39.580316,46.846155
3,茄类,2630.430804,3.000000,16.101600,35.193163,37.712348
4,辣椒类,6373.861663,3.000000,18.732000,100.551605,110.788459
5,食用菌,414.788814,1.017686,10.923144,110.185503,121.684707
